# 사고다발구역 파일 불러오기

In [1]:
import pandas as pd

df = pd.DataFrame()

df_total = pd.DataFrame()
for i in range(1,4,1):
    file_path = './data/{}.csv'.format(i)
    df = pd.read_csv(file_path, encoding='utf-8')
    df_total = pd.concat([df_total, df])
df

,Unnamed: 0,freqocZoneVer,freqocZoneId,freqocZoneNm,signguCode,signguNm,acdntCo,centerX,centerY,zoneRds
0,0,2021090301,1154500355,베스트마트,11545,서울특별시 금천구,3,126.903142,37.472332,40
1,1,2021090301,1154500356,신흥빌딩입구,11545,서울특별시 금천구,7,126.898679,37.477510,40
2,2,2021090301,1154500357,엘림데이케어센터,11545,서울특별시 금천구,6,126.901707,37.466493,40
3,3,2021090301,1154500358,M모텔,11545,서울특별시 금천구,4,126.903609,37.476282,40
4,4,2021090301,1154500359,아마존,11545,서울특별시 금천구,13,126.903998,37.479899,40
...,...,...,...,...,...,...,...,...,...,...
240,19,2021090301,1129000182,위니아딤채,11290,서울특별시 성북구,8,127.012689,37.591435,40
241,20,2021090301,1129000183,제일사,11290,서울특별시 성북구,3,127.021402,37.601773,40
242,21,2021090301,1129000184,미니스톱,11290,서울특별시 성북구,11,127.017784,37.593502,40
243,22,2021090301,1129000185,들꽃이한의원,11290,서울특별시 성북구,3,127.053216,37.608757,40


In [12]:
df_total.columns=['다발구역버전','다발구역아이디','다발구역명','시군구코드','시군구명','사고건수','중심X','중심Y','구역반경','?']
df_total = df_total.iloc[:,:-1]
df_total['시군구코드'].unique()
df_total = df_total[df_total['다발구역아이디'].duplicated()==False]
df_total = df_total.reset_index(drop=True)
df_total['시군구명2'] = df_total['시군구명'].str.split(" ").str[1]
df_total['시군구명2'].unique()
#df_total.to_csv('./data/이륜차사고다발지역.csv')

array(['송파구', '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구', '강남구',
       '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구',
       '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구'], dtype=object)

# 어린이 안전구역 파일 불러오기

In [36]:
df_safe = pd.read_csv('./data/전국어린이보호구역표준데이터.csv', encoding = 'euc-kr')
df_safe['소재지지번주소'].str.split(" ").str[1]
df_safe['시도'] = df_safe['제공기관명'].str.split(" ").str[0]
df_safe['시군구'] = df_safe['제공기관명'].str.split(" ").str[1]
df_safe_seoul = df_safe[df_safe['시도']=='서울특별시']
df_safe_seoul = df_safe_seoul.reset_index(drop=True)
df_safe_seoul = df_safe_seoul[['시도','시군구','대상시설명','시설종류','소재지지번주소','위도','경도','CCTV설치여부']]
df_safe_seoul = df_safe_seoul[df_safe_seoul['위도'].isnull()==False]
df_safe_seoul = df_safe_seoul[df_safe_seoul['경도'].isnull()==False]
df_safe_seoul = df_safe_seoul.reset_index(drop=True)

In [37]:
df_safe_seoul['여부']='Y'
df_safe_seoul.head()

,시도,시군구,대상시설명,시설종류,소재지지번주소,위도,경도,CCTV설치여부,여부
0,서울특별시,성북구,반디유치원,유치원,서울특별시 성북구 돈암동 609-1,37.594473,127.012706,Y,Y
1,서울특별시,성북구,사랑유치원,유치원,서울특별시 성북구 종암동 99-10,37.599900,127.036350,Y,Y
2,서울특별시,성북구,세아유치원,유치원,서울특별시 성북구 돈암동 609-1,37.594473,127.012706,Y,Y
3,서울특별시,성북구,은영유치원,유치원,서울특별시 성북구 보문동3가 168-2,37.584623,127.016684,Y,Y
4,서울특별시,성북구,탑주유치원,유치원,서울특별시 성북구 하월곡동 22,37.604780,127.041073,Y,Y


# 시군구별 사고건수 df

In [4]:
df_total = df_total[['사고건수','시군구명2']]
df_total_filt = df_total.groupby(['시군구명2']).sum()
df_total_filt = df_total_filt.reset_index()

In [5]:
df_total_filt.columns = ['sigungu','num']
df_total_filt.head(1)

,sigungu,num
0,강남구,316


# 지도 시각화

In [6]:
import numpy as np
import pandas as pd
import folium
import json

In [7]:
import warnings

warnings.filterwarnings(action='ignore')

In [8]:
#한글깨짐 해결
!pip3 install git+https://github.com/python-visualization/branca.git@master

  Cloning https://github.com/python-visualization/branca.git (to revision master) to c:\users\sjbang\appdata\local\temp\pip-req-build-zeqdmzj_
  Resolved https://github.com/python-visualization/branca.git to commit 52b10bd969ac25d0b70640b2eee2ea1d8e8690b3


  Running command git clone -q https://github.com/python-visualization/branca.git 'C:\Users\sjbang\AppData\Local\Temp\pip-req-build-zeqdmzj_'


### 어린이보호구역

In [9]:
from folium.features import CustomIcon

m = folium.Map(
    [37.554574, 126.979977],
    zoom_start = 10
)

file_path = "./geojson/서울_시군구.geojson"
seoul_sgg_get = json.load(open(file_path, encoding="utf-8"))
seoul_sgg_get

#print(len(seoul_sgg_get['features']))
      
for i in range (len(seoul_sgg_get['features'])):
    result = seoul_sgg_get['features'][i]['properties']
    print(result)

m = folium.Map(
    location=[37.554574, 126.979977],
    #tiles = 'Stamen Terrain',
    zoom_start=10   
)

# style 지정
def style_function(feature) :
    df = {"opacity" : 0.5,
        "weight" : 0.6,
        "color" : "gray",
        "fillOpacity" : 0.5}

    return df
    
folium.GeoJson(
    seoul_sgg_get,
    style_function = style_function
).add_to(m)

tooltip = 'Click!'


for i in df_safe_seoul.index:
    if df_safe_seoul.iloc[i]["시군구"] == "송파구":
    
        icon_image = 'children.png'
        icon = CustomIcon(
            icon_image,
            icon_size = (20, 20),
            popup_anchor = (30, -30)
        )

        folium.Marker(
            [df_safe_seoul.iloc[i]["위도"], df_safe_seoul.iloc[i]["경도"]],
            tooltip = tooltip,
            icon = icon
        ).add_to(m)

{'SIG_CD': '11110', 'SIG_ENG_NM': 'Jongno-gu', 'SIG_KOR_NM': '종로구'}
{'SIG_CD': '11140', 'SIG_ENG_NM': 'Jung-gu', 'SIG_KOR_NM': '중구'}
{'SIG_CD': '11170', 'SIG_ENG_NM': 'Yongsan-gu', 'SIG_KOR_NM': '용산구'}
{'SIG_CD': '11200', 'SIG_ENG_NM': 'Seongdong-gu', 'SIG_KOR_NM': '성동구'}
{'SIG_CD': '11215', 'SIG_ENG_NM': 'Gwangjin-gu', 'SIG_KOR_NM': '광진구'}
{'SIG_CD': '11230', 'SIG_ENG_NM': 'Dongdaemun-gu', 'SIG_KOR_NM': '동대문구'}
{'SIG_CD': '11260', 'SIG_ENG_NM': 'Jungnang-gu', 'SIG_KOR_NM': '중랑구'}
{'SIG_CD': '11290', 'SIG_ENG_NM': 'Seongbuk-gu', 'SIG_KOR_NM': '성북구'}
{'SIG_CD': '11305', 'SIG_ENG_NM': 'Gangbuk-gu', 'SIG_KOR_NM': '강북구'}
{'SIG_CD': '11320', 'SIG_ENG_NM': 'Dobong-gu', 'SIG_KOR_NM': '도봉구'}
{'SIG_CD': '11350', 'SIG_ENG_NM': 'Nowon-gu', 'SIG_KOR_NM': '노원구'}
{'SIG_CD': '11380', 'SIG_ENG_NM': 'Eunpyeong-gu', 'SIG_KOR_NM': '은평구'}
{'SIG_CD': '11410', 'SIG_ENG_NM': 'Seodaemun-gu', 'SIG_KOR_NM': '서대문구'}
{'SIG_CD': '11440', 'SIG_ENG_NM': 'Mapo-gu', 'SIG_KOR_NM': '마포구'}
{'SIG_CD': '11470', 'SIG_ENG_NM

### 히트맵으로 표시

In [13]:
m = folium.Map(
    [37.554574, 126.979977],
    zoom_start = 10
)

from folium import plugins

# latitude, longitude = 35.9078, 127.7669 #대한민국 위도, 경도
# s_korea = folium.Map(location = [latitude, longitude], zoom_start=8)
# s_korea
for i in df_total.index:
    if df_total.iloc[i]["시군구명2"] == "송파구":
        m.add_child(plugins.HeatMap(zip(df_total['중심Y'],
                                      df_total['중심X'],
                                      df_total['사고건수']), radius=18))

m

### 최종 지도

In [14]:
import plotly.express as px
import pandas as pd 
import os 
import json 

In [42]:
geometry_gj = json.load(open('./geojson/서울_시군구.geojson', encoding='utf-8'))

tooltip = 'Click!'

m = folium.Map(
    [37.554574, 126.979977],
    zoom_start = 10
)

folium.TileLayer('openstreetmap').add_to(m)
folium.TileLayer('Stamen Terrain').add_to(m)
folium.TileLayer('cartodbpositron').add_to(m)



fg=folium.FeatureGroup(name='My Points', show=False)

folium.Choropleth(
    geo_data=geometry_gj,
    data=df_total_filt, 
    columns=('sigungu','num'), 
    key_on='feature.properties.SIG_KOR_NM', 
    fill_color='BuPu',
    legend_name='이륜차 사고건수',
    ).add_to(m)
    
for i in df_total.index:
    if df_total.iloc[i]["시군구명2"] == "송파구":
        folium.Marker(
            [df_total.loc[i, "중심Y"], df_total.loc[i, "중심X"]],
            popup=f'<div style="width:80px"><strong>{df_total.iloc[i]["다발구역명"]}</strong><br>\
                        사고건수 : {df_total.iloc[i]["사고건수"]}</div>',
            tooltip = tooltip
        ).add_to(m)
        
        folium.CircleMarker(
            [df_total.loc[i, "중심Y"], df_total.loc[i, "중심X"]],
            color = 'red',
            fill = True,
            radius=df_total.iloc[i]['사고건수']/0.5,
            tooltip = tooltip
        ).add_to(m)
        
from folium.features import CustomIcon

어린이보호구역 = folium.FeatureGroup(name='어린이보호구역').add_to(m)


for i in df_safe_seoul.index:
    if df_safe_seoul.iloc[i]["시군구"] == "송파구":
        if df_safe_seoul.iloc[i]["여부"] == "Y":
            icon_image = 'children.png'
            icon = CustomIcon(
                icon_image,
                icon_size = (20, 20),
                popup_anchor = (30, -30)
            )

            folium.Marker(
                [df_safe_seoul.iloc[i]['위도'], df_safe_seoul.iloc[i]['경도']],
                popup=f'<div style="width:80px"><strong>{df_safe_seoul.iloc[i]["대상시설명"]}</strong></div>',
                icon=icon,
                tooltip = tooltip
            ).add_to(어린이보호구역)


folium.LayerControl(collapsed=False).add_to(m)

m.save('./html/송파구.html')

In [29]:
# for i in df_safe_seoul.index:
#     if df_safe_seoul.iloc[i]["시군구"] == "송파구":
    
#         icon_image = 'children.png'
#         icon = CustomIcon(
#             icon_image,
#             icon_size = (20, 20),
#             popup_anchor = (30, -30)
#         )

#         folium.Marker(
#             [df_safe_seoul.iloc[i]["위도"], df_safe_seoul.iloc[i]["경도"]],
#             tooltip = tooltip,
#             icon = icon
#         ).add_to(m)